In [ ]:
import os
import pandas as pd

BASE_DIR = "./../cohort"

PATH_129 = os.path.join(BASE_DIR, "cohort_ver129_target_log_transformed.csv")
PATH_183 = os.path.join(BASE_DIR, "cohort_ver183_add_delay_missing_flag.csv")
OUT_PATH = os.path.join(BASE_DIR, "cohort_ver151_129_183_merged_ordered.csv")

print("[LOAD] 129번 event-level 로딩...")
df129 = pd.read_csv(PATH_129)

print("[LOAD] 183번 static cohort 로딩...")
df183 = pd.read_csv(PATH_183)

# ----------------------------------------
# 1) 병합 키 설정: subject_id + hadm_id
# ----------------------------------------
join_keys = ["subject_id", "hadm_id"]

for k in join_keys:
    if k not in df129.columns or k not in df183.columns:
        raise ValueError(f"병합 키 {k}가 두 파일 모두에 존재해야 합니다.")

print("[INFO] 병합 키:", join_keys)

# ----------------------------------------
# 2) 183에서 129에 이미 존재하는 컬럼 제외 (키는 유지)
# ----------------------------------------
cols_183_to_add = [
    c for c in df183.columns
    if (c in join_keys) or (c not in df129.columns)
]

df183_sub = df183[cols_183_to_add].copy()

print(f"[INFO] 183에서 가져올 컬럼 수: {len(cols_183_to_add)}")

# ----------------------------------------
# 3) 병합 (129 기준 left join)
# ----------------------------------------
merged = df129.merge(df183_sub, on=join_keys, how="left")

print("[INFO] 병합 완료")
print(f" - 129 shape: {df129.shape}")
print(f" - 183 shape: {df183.shape}")
print(f" - merged shape: {merged.shape}")

# ----------------------------------------
# 4) door_to_*_missing_flag 처리
#    - 183에 door_to_ecg_missing 같은 이름이면 flag로 rename
#    - 그래도 없으면 door_to_* == -1 기준으로 생성
# ----------------------------------------
rename_map = {
    "door_to_ecg_missing":  "door_to_ecg_missing_flag",
    "door_to_trop_missing": "door_to_trop_missing_flag",
    "door_to_anti_missing": "door_to_anti_missing_flag",
    "door_to_pci_missing":  "door_to_pci_missing_flag",
}
# 4-1) 이름만 다른 컬럼 rename
for old, new in rename_map.items():
    if old in merged.columns and new not in merged.columns:
        merged = merged.rename(columns={old: new})

# 4-2) 여전히 flag 컬럼이 없으면 door_to_* 값으로부터 생성
for proc in ["ecg", "trop", "anti", "pci"]:
    flag_col = f"door_to_{proc}_missing_flag"
    base_col = f"door_to_{proc}"
    if flag_col not in merged.columns:
        if base_col in merged.columns:
            merged[flag_col] = (merged[base_col] == -1).astype(int)
        else:
            # door_to_* 자체도 없으면 그냥 0으로 채움 (missing 정보 없음)
            merged[flag_col] = 0

# ----------------------------------------
# 5) cum_stemi_cnt 생성 (없으면)
#    - hadm_id별로 stemi_flag 누적합
# ----------------------------------------
if "cum_stemi_cnt" not in merged.columns:
    if "stemi_flag" in merged.columns:
        merged = merged.sort_values(["hadm_id", "timestamp"])
        merged["cum_stemi_cnt"] = merged.groupby("hadm_id")["stemi_flag"].cumsum()
        print("[INFO] cum_stemi_cnt를 stemi_flag 누적합으로 생성했습니다.")
    else:
        merged["cum_stemi_cnt"] = 0
        print("[WARN] stemi_flag가 없어 cum_stemi_cnt를 0으로 생성했습니다.")

# ----------------------------------------
# 6) 최종 컬럼 순서 지정
# ----------------------------------------
final_cols = [
    "subject_id",                 # 1
    "hadm_id",                    # 2
    "age",                        # 3
    "gender",                     # 4
    "race",                       # 5
    "arrival_transport",          # 6
    "cci_score",                  # 7
    "hfrs_score",                 # 8
    "door_to_ecg",                # 9
    "door_to_trop",               # 10
    "door_to_anti",               # 11
    "door_to_pci",                # 12
    "door_to_ecg_missing_flag",   # 13
    "door_to_trop_missing_flag",  # 14
    "door_to_anti_missing_flag",  # 15
    "door_to_pci_missing_flag",   # 16
    "prefix_len",                 # 17
    "current_event_id",           # 18
    "time_since_start_min",       # 19
    "time_since_ed",              # 20
    "time_since_last",            # 21
    "is_night",                   # 22
    "cum_ecg_cnt",                # 23
    "cum_stemi_cnt",              # 24
    "cum_trop_cnt",               # 25
    "stemi_flag",                 # 26
    "trop_pos_flag",              # 27
    "last_trop",                  # 28
    "run_max_trop",               # 29
    "trop_trend",                 # 30
    "pci_status",                 # 31
    "current_heart_rate",         # 32
    "current_mean_bp",            # 33
    "timestamp",                  # 34
    "target_mortality",           # 35
    "target_next_evt",            # 36
    "target_time_to_next",        # 37
    "target_remain_los",          # 38
]

missing = [c for c in final_cols if c not in merged.columns]
if missing:
    raise ValueError(f"여전히 없는 컬럼: {missing}")

final_out = merged[final_cols].copy()

print("[DONE] 최종 컬럼 정렬 완료")
print(final_out.head())

# ----------------------------------------
# 7) 저장
# ----------------------------------------
final_out.to_csv(OUT_PATH, index=False)
print(f"[SAVE] {OUT_PATH} 에 저장 완료!")


[LOAD] 129번 event-level 로딩...
[LOAD] 183번 static cohort 로딩...
[INFO] 병합 키: ['subject_id', 'hadm_id']
[INFO] 183에서 가져올 컬럼 수: 12
[INFO] 병합 완료
 - 129 shape: (40817, 27)
 - 183 shape: (1929, 16)
 - merged shape: (40817, 37)
[INFO] cum_stemi_cnt를 stemi_flag 누적합으로 생성했습니다.
[DONE] 최종 컬럼 정렬 완료
   subject_id   hadm_id  age  gender                      race  \
0    13647833  20006266   40       1  ASIAN - SOUTH EAST ASIAN   
1    13647833  20006266   40       1  ASIAN - SOUTH EAST ASIAN   
2    13647833  20006266   40       1  ASIAN - SOUTH EAST ASIAN   
3    13647833  20006266   40       1  ASIAN - SOUTH EAST ASIAN   
4    13647833  20006266   40       1  ASIAN - SOUTH EAST ASIAN   

   arrival_transport  cci_score  hfrs_score  door_to_ecg  door_to_trop  ...  \
0                  1          0         0.0         -1.0         626.0  ...   
1                  1          0         0.0         -1.0         626.0  ...   
2                  1          0         0.0         -1.0         626.0  ...   
3